In [144]:
import requests
import json
import numpy as np
from tqdm.auto import tqdm
from collections import defaultdict
import pandas as pd
import nltk
import pymorphy2
from nltk.corpus import stopwords
import re
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import tree
from sklearn.metrics import mean_absolute_error as MAE

## Запрос вакансий с hh.ru

#### Выбор регионов (города-миллионники России)

In [2]:
cities = ['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 
          'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград']
regions = []
for city in cities:
    url = f'https://api.hh.ru/suggests/area_leaves?text={city}'
    res = requests.get(url)
    for item in res.json()['items']:
        if item['text'] in cities:
            regions.append(item)

In [3]:
regions

[{'id': '1', 'text': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 {'id': '2', 'text': 'Санкт-Петербург', 'url': 'https://api.hh.ru/areas/2'},
 {'id': '4', 'text': 'Новосибирск', 'url': 'https://api.hh.ru/areas/4'},
 {'id': '3', 'text': 'Екатеринбург', 'url': 'https://api.hh.ru/areas/3'},
 {'id': '88', 'text': 'Казань', 'url': 'https://api.hh.ru/areas/88'},
 {'id': '66', 'text': 'Нижний Новгород', 'url': 'https://api.hh.ru/areas/66'},
 {'id': '104', 'text': 'Челябинск', 'url': 'https://api.hh.ru/areas/104'},
 {'id': '54', 'text': 'Красноярск', 'url': 'https://api.hh.ru/areas/54'},
 {'id': '78', 'text': 'Самара', 'url': 'https://api.hh.ru/areas/78'},
 {'id': '99', 'text': 'Уфа', 'url': 'https://api.hh.ru/areas/99'},
 {'id': '76', 'text': 'Ростов-на-Дону', 'url': 'https://api.hh.ru/areas/76'},
 {'id': '68', 'text': 'Омск', 'url': 'https://api.hh.ru/areas/68'},
 {'id': '53', 'text': 'Краснодар', 'url': 'https://api.hh.ru/areas/53'},
 {'id': '26', 'text': 'Воронеж', 'url': 'https://api.h

#### Запрос списка вакансий по каждому региону

In [4]:
all_regions_vacs=[]
for region in tqdm(range(len(regions))):
    reg_id = regions[region]['id']
    for i in range(20):
        url = f'https://api.hh.ru/vacancies?page={i}&per_page=100&area={reg_id}&text=data'
        res = requests.get(url)
        vacancies = res.json()
        all_regions_vacs.extend(vacancies.get('items'))

  0%|          | 0/16 [00:00<?, ?it/s]

In [5]:
len(all_regions_vacs)

3322

#### Очистка параметров, которые не требуются для анализа

In [6]:
for vac in tqdm(range(len(all_regions_vacs))):
    for key in list(all_regions_vacs[vac].keys()):
        if 'url' in key:
            all_regions_vacs[vac].pop(key)
        if key=='address':
            all_regions_vacs[vac].pop(key)
        if key=='insider_interview':
            all_regions_vacs[vac].pop(key)            
    all_regions_vacs[vac]['area']=all_regions_vacs[vac]['area']['name']
    all_regions_vacs[vac]['employer']=all_regions_vacs[vac]['employer']['name']
    all_regions_vacs[vac]['type']=all_regions_vacs[vac]['type']['name']
    all_regions_vacs[vac]['schedule']=all_regions_vacs[vac]['schedule']['name']

  0%|          | 0/3322 [00:00<?, ?it/s]

In [7]:
all_regions_vacs[0]

{'id': '71426958',
 'premium': True,
 'name': 'Data Scientist (команда Поиска)',
 'department': {'id': 'hh-1455-ds',
  'name': 'HeadHunter::Analytics/Data Science'},
 'has_test': False,
 'response_letter_required': True,
 'area': 'Москва',
 'salary': {'from': 350000, 'to': None, 'currency': 'RUR', 'gross': True},
 'type': 'Открытая',
 'sort_point_distance': None,
 'published_at': '2022-10-27T15:35:01+0300',
 'created_at': '2022-10-27T15:35:01+0300',
 'archived': False,
 'relations': [],
 'employer': 'HeadHunter',
 'snippet': {'requirement': 'Опыт реализации и применения моделей машинного обучения в продакшн. Понимание основных методов и алгоритмов Machine Learning. Знание принципов работы нейросетевых...',
  'responsibility': 'Развитие моделей поискового ранжирования и рекомендательных систем. Тюнинг существующих моделей, feature engineering, а также построение новых решений с нуля. '},
 'contacts': None,
 'schedule': 'Полный день',
 'working_days': [],
 'working_time_intervals': [],
 

In [8]:
vacancies_table = pd.json_normalize(all_regions_vacs)
vacancies_table["published_at"]=pd.to_datetime(vacancies_table["published_at"])
vacancies_table["created_at"]=pd.to_datetime(vacancies_table["created_at"])
vacancies_table.dropna(how='all', axis=1, inplace=True)
vacancies_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3322 entries, 0 to 3321
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype                                
---  ------                    --------------  -----                                
 0   id                        3322 non-null   object                               
 1   premium                   3322 non-null   bool                                 
 2   name                      3322 non-null   object                               
 3   has_test                  3322 non-null   bool                                 
 4   response_letter_required  3322 non-null   bool                                 
 5   area                      3322 non-null   object                               
 6   type                      3322 non-null   object                               
 7   published_at              3322 non-null   datetime64[ns, pytz.FixedOffset(180)]
 8   created_at                3322 non-nul

In [9]:
vacancies_table.head()

,id,premium,name,has_test,response_letter_required,area,type,published_at,created_at,archived,...,working_time_modes,accept_temporary,department.id,department.name,salary.from,salary.to,salary.currency,salary.gross,snippet.requirement,snippet.responsibility
0,71426958,True,Data Scientist (команда Поиска),False,True,Москва,Открытая,2022-10-27 15:35:01+03:00,2022-10-27 15:35:01+03:00,False,...,[],False,hh-1455-ds,HeadHunter::Analytics/Data Science,350000.0,NaN,RUR,True,Опыт реализации и применения моделей машинного...,Развитие моделей поискового ранжирования и рек...
1,70994665,False,ML Team Lead (Играющий тренер data scientist),False,False,Москва,Открытая,2022-10-28 19:33:00+03:00,2022-10-28 19:33:00+03:00,False,...,[],False,NaN,NaN,750000.0,NaN,RUR,False,Глубокие знания в области Deep Learning/ Machi...,Основной проект компании - разработка унифицир...
2,71703435,False,Chief Data Officer (CDO),False,False,Москва,Открытая,2022-10-28 13:56:31+03:00,2022-10-28 13:56:31+03:00,False,...,[],False,NaN,NaN,3000.0,3700.0,EUR,True,The successful candidate must have a proven tr...,...and protecting <highlighttext>data</highlig...
3,70735115,False,Senior Data Scientist / ML Engineer,False,False,Москва,Открытая,2022-10-28 10:27:43+03:00,2022-10-28 10:27:43+03:00,False,...,[],False,NaN,NaN,300000.0,450000.0,RUR,False,"DS-stack: docker, python 3.7, sql, pandas, sci...","Детали. Нейронки, контент-бейзд рекомендации, ..."
4,71559745,False,Data scientist / ML разработчик,False,False,Москва,Открытая,2022-10-28 13:03:53+03:00,2022-10-28 13:03:53+03:00,False,...,[],False,NaN,NaN,120000.0,220000.0,RUR,False,Высшее образование в области прикладной матема...,"Участие в поддержке текущих, а также развертыв..."


#### Определение уровня вакансии по описанию

In [10]:
def vac_level(row):
    if 'Junior' in row['name']:
        return 'Junior'
    if 'Middle' in row['name']:
        return 'Middle'
    if ('Senior' in row['name'])|('Старший' in row['name']):
        return 'Senior'
    if ('Chief' in row['name'])|('Head' in row['name'])|('Lead' in row['name'])|('Руководитель' in row['name']):
        return 'Lead'

In [11]:
vacancies_table['name'].value_counts()

Data Engineer                                        68
Data Scientist                                       44
Data Analyst                                         27
Data engineer                                        25
Java-разработчик                                     25
                                                     ..
Frontend-разработчик (образовательная платформа)      1
HR Officer (British Embassy in Moscow, RUS22.474)     1
Dev-ops engineer                                      1
Middle Golang developer                               1
Frontend-разработчик Vue                              1
Name: name, Length: 2383, dtype: int64

In [12]:
vacancies_table['Level'] = vacancies_table.apply(lambda row: vac_level(row), axis=1)
vacancies_table

,id,premium,name,has_test,response_letter_required,area,type,published_at,created_at,archived,...,accept_temporary,department.id,department.name,salary.from,salary.to,salary.currency,salary.gross,snippet.requirement,snippet.responsibility,Level
0,71426958,True,Data Scientist (команда Поиска),False,True,Москва,Открытая,2022-10-27 15:35:01+03:00,2022-10-27 15:35:01+03:00,False,...,False,hh-1455-ds,HeadHunter::Analytics/Data Science,350000.0,NaN,RUR,True,Опыт реализации и применения моделей машинного...,Развитие моделей поискового ранжирования и рек...,None
1,70994665,False,ML Team Lead (Играющий тренер data scientist),False,False,Москва,Открытая,2022-10-28 19:33:00+03:00,2022-10-28 19:33:00+03:00,False,...,False,NaN,NaN,750000.0,NaN,RUR,False,Глубокие знания в области Deep Learning/ Machi...,Основной проект компании - разработка унифицир...,Lead
2,71703435,False,Chief Data Officer (CDO),False,False,Москва,Открытая,2022-10-28 13:56:31+03:00,2022-10-28 13:56:31+03:00,False,...,False,NaN,NaN,3000.0,3700.0,EUR,True,The successful candidate must have a proven tr...,...and protecting <highlighttext>data</highlig...,Lead
3,70735115,False,Senior Data Scientist / ML Engineer,False,False,Москва,Открытая,2022-10-28 10:27:43+03:00,2022-10-28 10:27:43+03:00,False,...,False,NaN,NaN,300000.0,450000.0,RUR,False,"DS-stack: docker, python 3.7, sql, pandas, sci...","Детали. Нейронки, контент-бейзд рекомендации, ...",Senior
4,71559745,False,Data scientist / ML разработчик,False,False,Москва,Открытая,2022-10-28 13:03:53+03:00,2022-10-28 13:03:53+03:00,False,...,False,NaN,NaN,120000.0,220000.0,RUR,False,Высшее образование в области прикладной матема...,"Участие в поддержке текущих, а также развертыв...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,71415345,False,ETL-developer,False,False,Волгоград,Открытая,2022-10-21 10:20:15+03:00,2022-10-21 10:20:15+03:00,False,...,False,NaN,NaN,NaN,NaN,NaN,NaN,"Опыт работы с NoSQL (сlickhouse, cassandra, el...",Изучать возможности разных ETL-систем и приним...,None
3318,71145212,False,Тайный покупатель (оператор проверки объявлени...,False,False,Волгоград,Открытая,2022-10-14 13:09:02+03:00,2022-10-14 13:09:02+03:00,False,...,True,NaN,NaN,NaN,22500.0,RUR,False,Готовы открыть самозанятость и работать по дог...,Обзвон объявлений по недвижимости в качестве т...,None
3319,71119665,False,Frontend-разработчик Vue,True,False,Волгоград,Открытая,2022-10-13 18:05:51+03:00,2022-10-13 18:05:51+03:00,False,...,False,NaN,NaN,90000.0,160000.0,RUR,False,Ищем фронтменов для текущих и новых проектов. ...,None,None
3320,69015080,False,Менеджер по работе с клиентами,False,False,Волгоград,Открытая,2022-10-12 15:17:42+03:00,2022-10-12 15:17:42+03:00,False,...,False,NaN,NaN,45000.0,NaN,RUR,True,"Мы обучим всему, что необходимо. Единственное,...",Вести телефонные переговоры с клиентами с цель...,None


In [13]:
vacancies_table['Level'].value_counts()

Senior    421
Lead      278
Middle    157
Junior     54
Name: Level, dtype: int64

#### Добавляем ключевые навыки

In [14]:
#Скачиваем описания всех вакансий
all_skills = []
descriptions = []
all_id = list(vacancies_table['id'])
for i in tqdm(range(len(all_id))):
    url = f'https://api.hh.ru/vacancies/{all_id[i]}'
    res = requests.get(url)
    all_skills.append(res.json().get('key_skills'))
    descriptions.append(res.json().get('description'))

  0%|          | 0/3322 [00:00<?, ?it/s]

In [15]:
url = f'https://api.hh.ru/vacancies/71426958'
res = requests.get(url)
res.json()

{'id': '71426958',
 'premium': True,
 'billing_type': {'id': 'premium', 'name': 'Премиум'},
 'relations': [],
 'name': 'Data Scientist (команда Поиска)',
 'insider_interview': None,
 'response_letter_required': True,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': {'from': 350000, 'to': None, 'currency': 'RUR', 'gross': True},
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Москва',
  'street': 'улица Годовикова',
  'building': '9с10',
  'description': None,
  'lat': 55.809343,
  'lng': 37.628505,
  'raw': 'Москва, улица Годовикова, 9с10',
  'metro': {'station_name': 'Алексеевская',
   'line_name': 'Калужско-Рижская',
   'station_id': '6.8',
   'line_id': '6',
   'lat': 55.807794,
   'lng': 37.638699},
  'metro_stations': [{'station_name': 'Алексеевская',
    'line_name': 'Калужско-Рижская',
    'station_id': '6.8',
    'line_id': '6',
    'lat': 55.807794,
    'lng': 37.638699}]},
 'allow_messages': True,
 'experience': {'id': 

In [16]:
skill_list = []
for skill in all_skills:
    if skill:
        temp_arr = []
        for item in skill:
            temp_arr.append(item["name"])
        skill_list.append(temp_arr)
    else: skill_list.append(False)

In [17]:
#Собираем навыки в объект Series, чтобы добавить колонкой к датафрейму
skills_table=pd.Series(skill_list)
skills_table.head(3)

0    [Python, PyTorch, SQL, Machine Learning, Анали...
1    [Python, PyTorch, Tensorflow, Deep Learning, K...
2    [Python, SQL, Английский язык, Leadership Skil...
dtype: object

In [18]:
#Добавляем колонку к датафрейму
vacs_and_skills=vacancies_table.copy(deep = True)
vacs_and_skills.insert(column="skills", value = skills_table,loc = 3)
vacs_and_skills.insert(column="description", value = pd.Series(descriptions),loc = 4)
vacs_and_skills.head()

,id,premium,name,skills,description,has_test,response_letter_required,area,type,published_at,...,accept_temporary,department.id,department.name,salary.from,salary.to,salary.currency,salary.gross,snippet.requirement,snippet.responsibility,Level
0,71426958,True,Data Scientist (команда Поиска),"[Python, PyTorch, SQL, Machine Learning, Анали...",<p>Мы ищем data scientist’а с опытом работы в ...,False,True,Москва,Открытая,2022-10-27 15:35:01+03:00,...,False,hh-1455-ds,HeadHunter::Analytics/Data Science,350000.0,NaN,RUR,True,Опыт реализации и применения моделей машинного...,Развитие моделей поискового ранжирования и рек...,None
1,70994665,False,ML Team Lead (Играющий тренер data scientist),"[Python, PyTorch, Tensorflow, Deep Learning, K...",<p>ООО &quot;Нейросетевая корпорация ЭкоСфера&...,False,False,Москва,Открытая,2022-10-28 19:33:00+03:00,...,False,NaN,NaN,750000.0,NaN,RUR,False,Глубокие знания в области Deep Learning/ Machi...,Основной проект компании - разработка унифицир...,Lead
2,71703435,False,Chief Data Officer (CDO),"[Python, SQL, Английский язык, Leadership Skil...",<p>We are an established b2b events organizing...,False,False,Москва,Открытая,2022-10-28 13:56:31+03:00,...,False,NaN,NaN,3000.0,3700.0,EUR,True,The successful candidate must have a proven tr...,...and protecting <highlighttext>data</highlig...,Lead
3,70735115,False,Senior Data Scientist / ML Engineer,"[Docker, Python, SQL, Pandas, Numpy, Scipy, Sc...",<p><strong>Mindbox</strong> — крупнейшая в Рос...,False,False,Москва,Открытая,2022-10-28 10:27:43+03:00,...,False,NaN,NaN,300000.0,450000.0,RUR,False,"DS-stack: docker, python 3.7, sql, pandas, sci...","Детали. Нейронки, контент-бейзд рекомендации, ...",Senior
4,71559745,False,Data scientist / ML разработчик,"[Python, SQL, A/B тесты, Airflow, Spark, Flask...",<p><strong>Компания Loymax - лидер на рынке IT...,False,False,Москва,Открытая,2022-10-28 13:03:53+03:00,...,False,NaN,NaN,120000.0,220000.0,RUR,False,Высшее образование в области прикладной матема...,"Участие в поддержке текущих, а также развертыв...",None


#### Нормализуем текст в описании вакансий, удаляем лишние символы

In [19]:
def cleaner(text):
    if text == None:
        return 'Empty'
    clean_text = re.sub(re.compile('<.*?>'), '', text)
    clean_text = re.sub(re.compile('\&.*?;'), '', clean_text).lower()
    return clean_text

In [51]:
vacs_and_skills['description'] = vacs_and_skills['description'].apply(lambda x:cleaner(x))
vacs_and_skills

,id,premium,name,skills,description,has_test,response_letter_required,area,type,published_at,...,accept_temporary,department.id,department.name,salary.from,salary.to,salary.currency,salary.gross,snippet.requirement,snippet.responsibility,Level
0,71426958,True,Data Scientist (команда Поиска),"[Python, PyTorch, SQL, Machine Learning, Анали...",мы ищем data scientist’а с опытом работы в тек...,False,True,Москва,Открытая,2022-10-27 15:35:01+03:00,...,False,hh-1455-ds,HeadHunter::Analytics/Data Science,350000.0,NaN,RUR,True,Опыт реализации и применения моделей машинного...,Развитие моделей поискового ранжирования и рек...,None
1,70994665,False,ML Team Lead (Играющий тренер data scientist),"[Python, PyTorch, Tensorflow, Deep Learning, K...",ооо нейросетевая корпорация экосфера занимаетс...,False,False,Москва,Открытая,2022-10-28 19:33:00+03:00,...,False,NaN,NaN,750000.0,NaN,RUR,False,Глубокие знания в области Deep Learning/ Machi...,Основной проект компании - разработка унифицир...,Lead
2,71703435,False,Chief Data Officer (CDO),"[Python, SQL, Английский язык, Leadership Skil...",we are an established b2b events organizing bu...,False,False,Москва,Открытая,2022-10-28 13:56:31+03:00,...,False,NaN,NaN,3000.0,3700.0,EUR,True,The successful candidate must have a proven tr...,...and protecting <highlighttext>data</highlig...,Lead
3,70735115,False,Senior Data Scientist / ML Engineer,"[Docker, Python, SQL, Pandas, Numpy, Scipy, Sc...",mindbox — крупнейшая в россии облачная платфор...,False,False,Москва,Открытая,2022-10-28 10:27:43+03:00,...,False,NaN,NaN,300000.0,450000.0,RUR,False,"DS-stack: docker, python 3.7, sql, pandas, sci...","Детали. Нейронки, контент-бейзд рекомендации, ...",Senior
4,71559745,False,Data scientist / ML разработчик,"[Python, SQL, A/B тесты, Airflow, Spark, Flask...",компания loymax - лидер на рынке it решений по...,False,False,Москва,Открытая,2022-10-28 13:03:53+03:00,...,False,NaN,NaN,120000.0,220000.0,RUR,False,Высшее образование в области прикладной матема...,"Участие в поддержке текущих, а также развертыв...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,71415345,False,ETL-developer,"[SQL, Python, Java, PostgreSQL, ETL, Clickhous...",ищем коллегу на проект по созданию и развитию ...,False,False,Волгоград,Открытая,2022-10-21 10:20:15+03:00,...,False,NaN,NaN,NaN,NaN,NaN,NaN,"Опыт работы с NoSQL (сlickhouse, cassandra, el...",Изучать возможности разных ETL-систем и приним...,None
3318,71145212,False,Тайный покупатель (оператор проверки объявлени...,"[Консультирование клиентов по телефону, Коммун...",компания training data занимается разметкой да...,False,False,Волгоград,Открытая,2022-10-14 13:09:02+03:00,...,True,NaN,NaN,NaN,22500.0,RUR,False,Готовы открыть самозанятость и работать по дог...,Обзвон объявлений по недвижимости в качестве т...,None
3319,71119665,False,Frontend-разработчик Vue,"[JavaScript, Vue.js, HTML, CSS, Babel, Webpack...",justwork — аутсорс-разработчик систем со сложн...,True,False,Волгоград,Открытая,2022-10-13 18:05:51+03:00,...,False,NaN,NaN,90000.0,160000.0,RUR,False,Ищем фронтменов для текущих и новых проектов. ...,None,None
3320,69015080,False,Менеджер по работе с клиентами,"[Телефонные переговоры, Работа с большим объем...","idf eurasia специализируется на data science, ...",False,False,Волгоград,Открытая,2022-10-12 15:17:42+03:00,...,False,NaN,NaN,45000.0,NaN,RUR,True,"Мы обучим всему, что необходимо. Единственное,...",Вести телефонные переговоры с клиентами с цель...,None


#### Прогнозируем уровень вакансии по ее описанию

In [112]:
leveled_vacs = vacs_and_skills[vacs_and_skills['Level'].notnull()].copy()
not_leveled_vacs = vacs_and_skills[vacs_and_skills['Level'].isnull()].copy()
leveled_vacs['Level'].unique()

array(['Lead', 'Senior', 'Junior', 'Middle'], dtype=object)

In [53]:
nltk.download('stopwords')
stop_russian = stopwords.words('russian')


[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [79]:
text_transformer = TfidfVectorizer(stop_words=stop_russian)
description = text_transformer.fit_transform(leveled_vacs['description'])


In [113]:
levels_dict = pd.Series(leveled_vacs['Level'].unique()).to_dict()
levels_dict = {y: x for x, y in levels_dict.items()}
leveled_vacs['Level'] = leveled_vacs['Level'].apply(lambda x:levels_dict[x])

In [91]:
X_train, X_test, y_train, y_test = train_test_split(description, leveled_vacs['Level'], test_size=0.20, random_state=123)
clf = tree.DecisionTreeClassifier()
level_fit = clf.fit(X_train,y_train)

In [92]:
pred = level_fit.predict(X_test)
print('Качество модели по метрике F1', f1_score(y_test,pred,average='weighted'))

Качество модели по метрике F1 0.6685033860149067


In [87]:
not_leveled_vacs = vacs_and_skills[vacs_and_skills['Level'].isnull()].copy()
description_nl = text_transformer.fit_transform(not_leveled_vacs['description'])

In [99]:
description

<910x22320 sparse matrix of type '<class 'numpy.float64'>'
	with 177631 stored elements in Compressed Sparse Row format>

In [105]:
len(level_fit.predict(description_nl[:, :22320]))

2412

In [115]:
levels_dict = {y: x for x, y in levels_dict.items()}

In [116]:
levels_dict

{0: 'Lead', 1: 'Senior', 2: 'Junior', 3: 'Middle'}

In [140]:
predicted_level = pd.Series(level_fit.predict(description_nl[:, :22320]))
predicted_level = predicted_level.apply(lambda x:levels_dict[x])
predicted_level = pd.DataFrame(predicted_level)
predicted_level['id'] = vacs_and_skills[vacs_and_skills['Level'].isnull()].reset_index()['id']
predicted_level

,0,id
0,Senior,71426958
1,Senior,71559745
2,Senior,71209351
3,Senior,71714632
4,Senior,71562226
...,...,...
2407,Senior,71315359
2408,Senior,71415345
2409,Senior,71145212
2410,Senior,71119665


In [162]:
vacs_and_skills = vacs_and_skills.join(predicted_level.set_index('id'), on='id')

In [163]:
vacs_and_skills['Level'].fillna(vacs_and_skills[0], inplace=True)
vacs_and_skills.drop(labels = [0], axis = 1, inplace = True)
vacs_and_skills

,id,premium,name,skills,description,has_test,response_letter_required,area,type,published_at,...,accept_temporary,department.id,department.name,salary.from,salary.to,salary.currency,salary.gross,snippet.requirement,snippet.responsibility,Level
0,71426958,True,Data Scientist (команда Поиска),"[Python, PyTorch, SQL, Machine Learning, Анали...",мы ищем data scientist’а с опытом работы в тек...,False,True,Москва,Открытая,2022-10-27 15:35:01+03:00,...,False,hh-1455-ds,HeadHunter::Analytics/Data Science,350000.0,NaN,RUR,True,Опыт реализации и применения моделей машинного...,Развитие моделей поискового ранжирования и рек...,Senior
1,70994665,False,ML Team Lead (Играющий тренер data scientist),"[Python, PyTorch, Tensorflow, Deep Learning, K...",ооо нейросетевая корпорация экосфера занимаетс...,False,False,Москва,Открытая,2022-10-28 19:33:00+03:00,...,False,NaN,NaN,750000.0,NaN,RUR,False,Глубокие знания в области Deep Learning/ Machi...,Основной проект компании - разработка унифицир...,Lead
2,71703435,False,Chief Data Officer (CDO),"[Python, SQL, Английский язык, Leadership Skil...",we are an established b2b events organizing bu...,False,False,Москва,Открытая,2022-10-28 13:56:31+03:00,...,False,NaN,NaN,3000.0,3700.0,EUR,True,The successful candidate must have a proven tr...,...and protecting <highlighttext>data</highlig...,Lead
3,70735115,False,Senior Data Scientist / ML Engineer,"[Docker, Python, SQL, Pandas, Numpy, Scipy, Sc...",mindbox — крупнейшая в россии облачная платфор...,False,False,Москва,Открытая,2022-10-28 10:27:43+03:00,...,False,NaN,NaN,300000.0,450000.0,RUR,False,"DS-stack: docker, python 3.7, sql, pandas, sci...","Детали. Нейронки, контент-бейзд рекомендации, ...",Senior
4,71559745,False,Data scientist / ML разработчик,"[Python, SQL, A/B тесты, Airflow, Spark, Flask...",компания loymax - лидер на рынке it решений по...,False,False,Москва,Открытая,2022-10-28 13:03:53+03:00,...,False,NaN,NaN,120000.0,220000.0,RUR,False,Высшее образование в области прикладной матема...,"Участие в поддержке текущих, а также развертыв...",Senior
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,71415345,False,ETL-developer,"[SQL, Python, Java, PostgreSQL, ETL, Clickhous...",ищем коллегу на проект по созданию и развитию ...,False,False,Волгоград,Открытая,2022-10-21 10:20:15+03:00,...,False,NaN,NaN,NaN,NaN,NaN,NaN,"Опыт работы с NoSQL (сlickhouse, cassandra, el...",Изучать возможности разных ETL-систем и приним...,Senior
3318,71145212,False,Тайный покупатель (оператор проверки объявлени...,"[Консультирование клиентов по телефону, Коммун...",компания training data занимается разметкой да...,False,False,Волгоград,Открытая,2022-10-14 13:09:02+03:00,...,True,NaN,NaN,NaN,22500.0,RUR,False,Готовы открыть самозанятость и работать по дог...,Обзвон объявлений по недвижимости в качестве т...,Senior
3319,71119665,False,Frontend-разработчик Vue,"[JavaScript, Vue.js, HTML, CSS, Babel, Webpack...",justwork — аутсорс-разработчик систем со сложн...,True,False,Волгоград,Открытая,2022-10-13 18:05:51+03:00,...,False,NaN,NaN,90000.0,160000.0,RUR,False,Ищем фронтменов для текущих и новых проектов. ...,None,Senior
3320,69015080,False,Менеджер по работе с клиентами,"[Телефонные переговоры, Работа с большим объем...","idf eurasia специализируется на data science, ...",False,False,Волгоград,Открытая,2022-10-12 15:17:42+03:00,...,False,NaN,NaN,45000.0,NaN,RUR,True,"Мы обучим всему, что необходимо. Единственное,...",Вести телефонные переговоры с клиентами с цель...,Senior


#### Строим модель для прогнозирования зарплаты в вакансиях, где она не указана

In [164]:
df = vacs_and_skills[vacs_and_skills['salary.from'].notnull()].copy()
df.drop(labels=['type','premium','archived','relations','working_days','published_at',
                'created_at','working_time_modes','working_time_intervals'],axis=1, inplace=True)
df['salary.currency'] = df['salary.currency'].map({'RUR':0,'USD':1,'EUR':2})
df['has_test']=df['has_test'].astype(int)
df['response_letter_required']=df['response_letter_required'].astype(int)
df['accept_temporary']=df['accept_temporary'].astype(int)
df['id']=df['id'].astype(int)
df['description'] = df['description'].fillna('Empty')
df['skills']=df['skills'].str.join(' ,').fillna('Empty')
df = df.dropna(axis=1)

In [165]:
df

,id,name,skills,description,has_test,response_letter_required,area,employer,schedule,accept_temporary,salary.from,salary.currency,Level
0,71426958,Data Scientist (команда Поиска),"Python ,PyTorch ,SQL ,Machine Learning ,Анализ...",мы ищем data scientist’а с опытом работы в тек...,0,1,Москва,HeadHunter,Полный день,0,350000.0,0,Senior
1,70994665,ML Team Lead (Играющий тренер data scientist),"Python ,PyTorch ,Tensorflow ,Deep Learning ,Keras",ооо нейросетевая корпорация экосфера занимаетс...,0,0,Москва,НСК ЭкоСфера,Полный день,0,750000.0,0,Lead
2,71703435,Chief Data Officer (CDO),"Python ,SQL ,Английский язык ,Leadership Skill...",we are an established b2b events organizing bu...,0,0,Москва,НАО Тринити Ивентс,Удаленная работа,0,3000.0,2,Lead
3,70735115,Senior Data Scientist / ML Engineer,"Docker ,Python ,SQL ,Pandas ,Numpy ,Scipy ,Sci...",mindbox — крупнейшая в россии облачная платфор...,0,0,Москва,Mindbox,Гибкий график,0,300000.0,0,Senior
4,71559745,Data scientist / ML разработчик,"Python ,SQL ,A/B тесты ,Airflow ,Spark ,Flask ...",компания loymax - лидер на рынке it решений по...,0,0,Москва,Loymax,Полный день,0,120000.0,0,Senior
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3314,71021916,Старший специалист по продажам,"Прямые продажи ,Деловое общение ,Активные прод...",билайн предоставляет услуги как крупнейшим меж...,0,0,Волгоград,билайн,Полный день,0,70000.0,0,Senior
3315,69841085,Специалист по работе с клиентами (Мани Мен),"Грамотная речь ,Работа с базами данных ,Работа...","id finance специализируется на data science, к...",0,0,Волгоград,IDF Eurasia,Полный день,0,40000.0,0,Senior
3316,71315359,Специалист технической поддержки в Яндекс Верт...,"HTTP ,Многозадачность ,API ,Умение расставлять...",cm.expert — первая и ведущая в россии data min...,1,0,Волгоград,Яндекс,Удаленная работа,0,28000.0,0,Senior
3319,71119665,Frontend-разработчик Vue,"JavaScript ,Vue.js ,HTML ,CSS ,Babel ,Webpack ...",justwork — аутсорс-разработчик систем со сложн...,1,0,Волгоград,ДЖАСТ ВОРК,Удаленная работа,0,90000.0,0,Senior


In [166]:
#Трансформируем название вакансии в матрицу TF-IDF
text_transformer = TfidfVectorizer()
names = text_transformer.fit_transform(df['name'])

In [167]:
#Трансформируем навыки в матрицу TF-IDF
skills = text_transformer.fit_transform(df['skills'])

In [168]:
#Трансформируем название организации-работодателя в матрицу TF-IDF
employer = text_transformer.fit_transform(df['employer'])

In [169]:
#Трансформируем описание вакансии в матрицу TF-IDF
text_transformer = TfidfVectorizer(stop_words=stop_russian)
description = text_transformer.fit_transform(df['description'])

#### Сравниваем метрику MAE при расчете по разным данным

In [170]:
clf = tree.DecisionTreeClassifier()

In [171]:
X_train, X_test, y_train, y_test = train_test_split(names, df['salary.from'], test_size=0.20, random_state=123)
names_fit = clf.fit(X_train,y_train)
print('Ошибка в предсказании по названию вакансии -', MAE(y_test, names_fit.predict(X_test)))
print('Средняя зарплата -', df['salary.from'].mean())

Ошибка в предсказании по названию вакансии - 47849.94160583941
Средняя зарплата - 103626.84875183554


In [172]:
X_train, X_test, y_train, y_test = train_test_split(skills, df['salary.from'], test_size=0.20, random_state=123)
skills_fit = clf.fit(X_train,y_train)
print('Ошибка в предсказании по навыкам -', MAE(y_test, skills_fit.predict(X_test)))
print('Средняя зарплата -', df['salary.from'].mean())

Ошибка в предсказании по навыкам - 50343.54744525548
Средняя зарплата - 103626.84875183554


In [173]:
X_train, X_test, y_train, y_test = train_test_split(employer, df['salary.from'], test_size=0.20, random_state=123)
emploer_fit = clf.fit(X_train,y_train)
print('Ошибка в предсказании по работодателю -', MAE(y_test, emploer_fit.predict(X_test)))
print('Средняя зарплата -', df['salary.from'].mean())

Ошибка в предсказании по работодателю - 59406.087591240874
Средняя зарплата - 103626.84875183554


In [174]:
X_train, X_test, y_train, y_test = train_test_split(description, df['salary.from'], test_size=0.20, random_state=123)
description_fit = clf.fit(X_train,y_train)
print('Ошибка в предсказании по описанию вакансии -', MAE(y_test, description_fit.predict(X_test)))
print('Средняя зарплата -', df['salary.from'].mean())

Ошибка в предсказании по описанию вакансии - 38153.53284671533
Средняя зарплата - 103626.84875183554


#### Строим модель линейной регрессии для анализа по нетекстовым колонкам

In [198]:
lr_df = df.drop(labels=['id','name','skills','description'], axis=1)
lr_df

,has_test,response_letter_required,area,employer,schedule,accept_temporary,salary.from,salary.currency,Level
0,0,1,Москва,HeadHunter,Полный день,0,350000.0,0,Senior
1,0,0,Москва,НСК ЭкоСфера,Полный день,0,750000.0,0,Lead
2,0,0,Москва,НАО Тринити Ивентс,Удаленная работа,0,3000.0,2,Lead
3,0,0,Москва,Mindbox,Гибкий график,0,300000.0,0,Senior
4,0,0,Москва,Loymax,Полный день,0,120000.0,0,Senior
...,...,...,...,...,...,...,...,...,...
3314,0,0,Волгоград,билайн,Полный день,0,70000.0,0,Senior
3315,0,0,Волгоград,IDF Eurasia,Полный день,0,40000.0,0,Senior
3316,1,0,Волгоград,Яндекс,Удаленная работа,0,28000.0,0,Senior
3319,1,0,Волгоград,ДЖАСТ ВОРК,Удаленная работа,0,90000.0,0,Senior


In [189]:
levels_dict = {y: x for x, y in levels_dict.items()}
levels_dict

{'Lead': 0, 'Senior': 1, 'Junior': 2, 'Middle': 3}

In [199]:
emp_dict = pd.Series(lr_df['employer'].unique()).to_dict()
emp_dict = {y: x for x, y in emp_dict.items()}

In [200]:
reg_dict = {}
for region in regions:
    reg_dict[region['text']] = region['id']

In [201]:
schedule_dict = pd.Series(lr_df['schedule'].unique()).to_dict()
schedule_dict = {y: x for x, y in schedule_dict.items()}

In [202]:
lr_df['employer'] = lr_df['employer'].apply(lambda x:emp_dict[x])
lr_df['schedule'] = lr_df['schedule'].apply(lambda x:schedule_dict[x])
lr_df['area'] = lr_df['area'].apply(lambda x:reg_dict[x])
lr_df['Level'] = lr_df['Level'].apply(lambda x:levels_dict[x])
lr_df

,has_test,response_letter_required,area,employer,schedule,accept_temporary,salary.from,salary.currency,Level
0,0,1,1,0,0,0,350000.0,0,1
1,0,0,1,1,0,0,750000.0,0,0
2,0,0,1,2,1,0,3000.0,2,0
3,0,0,1,3,2,0,300000.0,0,1
4,0,0,1,4,0,0,120000.0,0,1
...,...,...,...,...,...,...,...,...,...
3314,0,0,24,89,0,0,70000.0,0,1
3315,0,0,24,137,0,0,40000.0,0,1
3316,1,0,24,214,1,0,28000.0,0,1
3319,1,0,24,267,1,0,90000.0,0,1


In [203]:
from sklearn.linear_model import LinearRegression as lr
X_train, X_test, y_train, y_test = train_test_split(lr_df.drop('salary.from',axis=1),
                                                   lr_df['salary.from'],
                                                   test_size=0.2,
                                                   random_state=123
                                                   )

In [204]:
clf = lr()
lr_fit = clf.fit(X_train,y_train)
print('Ошибка в предсказании по линейной регрессии -', MAE(lr_fit.predict(X_test),y_test))
print('Средняя зарплата -', df['salary.from'].mean())

Ошибка в предсказании по линейной регрессии - 55172.42606915337
Средняя зарплата - 103626.84875183554
